In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import os
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader

In [2]:
def select_features(train_data, test_data):
    y_train, y_test  = train_data[:, -1], test_data[:, -1]
    x_train, x_test  = train_data[:, :-1], test_data[:, :-1]

    return x_train, x_test, y_train, y_test

def predict(model, x):
    y = model(x)
    

In [3]:
# 读数据的类
class myData(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        return self.x[index], self.y[index]

In [4]:
import torch
import torch.nn as nn

# network structure
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
    
    def forward(self, x):
        x = self.layers(x)
        return x.squeeze(1)

In [5]:
# 训练
def train(train_loader, valid_loader, config, device):
    criterion = nn.MSELoss()
    # 动态学习率
    optimizer = torch.optim.SGD(model.parameters(), lr = config['learning_rate'], momentum=0.9)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')
    
    n_epochs, best_loss, step, early_step_count = config['n_epochs'], float('inf'), 0, 0
    
    # 训练 交叉验证 验证
    # train valid test
    for epoch in range(n_epochs):
        model.train()
        loss_record = []
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            # train_pbar.set_postfix({'loss:', loss.detach().item()})
        
        mean_train_loss = sum(loss_record) / len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []

        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)
            loss_record.append(loss.item())
        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}] : Train loss : {mean_train_loss:.4f}, Valid loss : {mean_valid_loss: .4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])
            print('Saving model with loss {:3f}...'.format(best_loss))
            early_step_count = 0
        else:
            early_step_count += 1
        
        if early_step_count >= config['early_stop']:
            print('\nM<odel is not improving, so we halt the training session.')
            return 

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 超参数
config = {
    'seed' : 20020225,
    'select_all': True,
    'valid_ratio': 0.2,
    'n_epochs': 3000,
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,
    'save_path': './models/model.ckpt'
}

In [7]:
same_seed = config['seed']

train_data = pd.read_csv('ml2022spring-hw1/covid.train.csv').values
test_data = pd.read_csv('ml2022spring-hw1/covid.test.csv').values

x_train, x_test, y_train, y_test = select_features(train_data, test_data)

split_nums = int(config['valid_ratio'] * len(x_train))

x_train, x_valid = x_train[:split_nums], x_train[split_nums:-1]
y_train, y_valid = y_train[:split_nums], y_train[split_nums:-1]

train_dataset = myData(x_train, y_train)
valid_dataset = myData(x_valid, y_valid)
test_dataset = myData(x_test)

train_loader = DataLoader(train_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size = config['batch_size'], shuffle=True, pin_memory=True)

In [8]:
model = Model(input_dim=x_train.shape[1]).to(device)
train(train_loader, valid_loader, config, device)

Epoch [1/3000]: 100%|██████████| 3/3 [00:00<00:00, 199.94it/s]


Epoch [1/3000] : Train loss : 72.0382, Valid loss :  69.1617
Saving model with loss 69.161699...


Epoch [2/3000]: 100%|██████████| 3/3 [00:00<00:00, 420.17it/s]


Epoch [2/3000] : Train loss : 44.2455, Valid loss :  101.9081


Epoch [3/3000]: 100%|██████████| 3/3 [00:00<00:00, 447.28it/s]


...

Epoch [2460/3000] : Train loss : 1.3836, Valid loss :  3.4857


Epoch [2461/3000]: 100%|██████████| 3/3 [00:00<00:00, 439.67it/s]


Epoch [2461/3000] : Train loss : 1.4344, Valid loss :  9.5453


Epoch [2462/3000]: 100%|██████████| 3/3 [00:00<00:00, 471.99it/s]


Epoch [2462/3000] : Train loss : 1.4654, Valid loss :  4.3649

M<odel is not improving, so we halt the training session.
